<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

# Global DEMO version 2.2.1

## Example for OCR for clinical reports and text analysis

In many real use cases the raw content of electronic health records or laboratory results are stored in PDFs so the extraction of their content requires using some kind of OCR capability.

The SparkNLP framework provides with a ready-to-use component that will extract content from PDFs called OcrHelper().

In the following example we will extract the content from a clinical record stored in PDF, extract its sentences and generate a Part of Speech tagging specifically trained for biomedical content with deep learning techniques. We will also leverage a spell checker trained to correct specific medical domain content with the annotator ContextSpellCheckerModel (spellcheck_clinical). 

For this example we will use as input a fictitious medical record stored in PDF that has been taken from this URL: https://www.mlaw.gov.sg/content/dam/minlaw/lab/News%20Publications/Documents/Sample%20filled%20in%20MR.pdf

### Step 1. Prepare the environment

#### Install OpenSource spark-nlp and pyspark pip packages
As a first step we import the required python dependences including some sparknlp components.

Be sure that you have the required python libraries (pyspark 2.4.3, spark-nlp 2.2.1) by running <code>pip list</code>. Check that the versions are correct.

If some of them is missing you can run:

<code>pip install --ignore-installed pyspark==2.4.3</code><br>
<code>pip install --ignore-installed spark-nlp==2.2.1</code><br>

The --ignore-installed parameter is to overwrite your previous pip package version if already installed.

<i>*If this cell fails means you have not propertly setup the required environment. Please check the pre-requisites guideline at http://www.johnsnowlabs.com</i>

In [2]:
import sys, time, os

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *
from sparknlp.embeddings import *
from sparknlp.ocr import *

from sparknlp.embeddings import EmbeddingsHelper
from sparknlp.pretrained import ResourceDownloader

from pyspark.ml import Pipeline, PipelineModel



#### Install Licensed Sparl-NLP package

We will use also some Spark-NLP enterprise functionalities contained in the spark-nlp-jsl package.

You can check that spark-nlp-jsl is installed by running <code>pip install</code>. Check that version installed is 2.2.1.

If it is not then you need to install it by using:

<code>pip install spark-nlp-jsl==2.2.1 --extra-index-url #### --ignore-installed</code>

The ####### is a secret url, if you have not received it please contact us at info@johnsnowlabs.com.

<i>*If the next cell fails means your licensed enterprise version is not propertly installed so please check the pre-requisites guideline at http://www.johnsnowlabs.com/</i>

In [3]:
# If this fails, means pip module for enterprise has not been properly set up

from sparknlp_jsl.annotator import *

#### Setup credentials to private JohnSnowLabs models repository with AWS-CLI

Now is time to configure Spark-NLP in order to access private JohnSnowLabs models repository. This access is done via Amazon aws command line interface (AWSCLI).

Instructions about how to install awscli are available at: 

https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html

Make sure you configure your credentials with <code>aws configure</code> following the instructions at:

https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html

Please substitute the ACCESS_KEY and SECRET_KEY with the credentials you have recived. If you need your credentials contact us at info@johnsnowlabs.com

#### Start Spark session

The following will initialize the spark session in case you have run the jupyter notebook directly. If you have started the notebook using pyspark this cell is just ignored.

Initializing the spark session takes some seconds (usually less than 1 minute) as the jar from the server needs to be loaded.

We will be using version 2.2.1 of Spark NLP Open Source and 2.2.1 of Spark NLP Enterprise Edition.

The ####### in <code>.config("spark.jars", "####")</code> is a secret URL, if you have not received it please contact us at info@johnsnowlabs.com.

In [4]:
# This cell will be ignored if jupyter started using pyspark
# This cell could take a while to run so be patient.

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Global DEMO - Spark NLP Enterprise 2.2.1") \
    .master("local[*]") \
    .config("spark.driver.memory","4G") \
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.2.1,com.johnsnowlabs.nlp:spark-nlp-ocr_2.11:2.2.1,javax.media.jai:com.springsource.javax.media.jai.core:1.1.3") \
    .config("spark.jars.repositories", "http://repo.spring.io/plugins-release") \
    .config("spark.jars", "####") \
    .getOrCreate()


# Step 2. Extract content from PDF with OCR

We will use the OcrHelper() that will extract the content of our sample_medical_record.pdf and store it in a Spark Dataframe.

In [5]:
data = OcrHelper().createDataset(spark=spark, input_path="./sample_medical_record.pdf")

Once the content is extracted from the PDF we can visualize for example the second page of this PDF.

In [6]:
print(data.select('text').take(2)[1]['text'])

- 2 - 
 
 
Doctor-patient relationship: 
 
Please state if you have been seeing the patient regularly over a period of time (if so, 
please state when you first started seeing the patient and how often you see the 
patient) or if you saw the patient specifically for this mental capacity assessment only. 
 
 
I have been the doctor in charge of Mr Tan Ah Kow since November 2010. I have 
seen Mr Tan regularly since then, on average once or twice a year. For purposes of 
this medical report, I re-examined Mr Tan on 20 June 2015. 
 
 
 
 
 
 
 
 
 
SECTION 3: PATIENT’S MEDICAL INFORMATION 
 
 
Patient’s clinical history: 
 
Please also state the source of the information (e.g. from medical records, from the 
patient, from the applicant etc.). 
 
Mr Tan Ah Kow was accompanied by his son, Mr Tan Ah Beng, for the examination. 
Mr Tan is a 55 year old man, who is divorced, and unemployed. Mr Tan is currently 
living with his son, Ah Beng, in Ah Beng’s flat. Mr Tan Ah Beng informed me that 
Mr 

### Step 3. Create the pipeline

Our pipeline's architecture will be composed by N annotators working sequentially:

* DocumentAssembler (that will take some text and return a document).
* SentenceDetector (that will detect sentence boundaries generating a list of sentences from a given document)
* Tokenizer (that will producr tokens from the sentences)
* PerceptronModel pretrained "pos_clinical" (that is a pretrained Part-Of-Speech tagger using an specific medical domain corpus.
* ContextSpellCheckerModel pretrained "spellcheck_clinical" (that is a pretrained Spellchecker based on biomedical training corpus)



In [7]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence") \
  .setExplodeSentences(True)

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

Be aware that the pretrained models (PerceptronModel and ContextSpellCheckerModel) can take a while to run, specially if you are executing them for the first time, as the models need to be downloaded, usually in your ~HOME directory at:

<code>~HOME/cache_pretrained</code>

The POS tagging model has a size of 1.7Mb and the SpellChecker a size of 128Mb.

In [8]:
# Named Entity Recognition for clinical concepts. Includes #Problems #Diagnostics

clinical_pos = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token"]) \
  .setOutputCol("pos")

pos_clinical download started this may take some time.
Approximate size to download 1.7 MB
[OK!]


In [9]:
# Assertion Status, verifies whether a particular subject wears a condition or not, and labels the condition by status


clinical_spell = ContextSpellCheckerModel.pretrained("context_spell_med", "en", "clinical/models") \
  .setInputCols(["token"]) \
  .setOutputCol("checked")
  


context_spell_med download started this may take some time.
Approximate size to download 117 MB
[OK!]


Now we can "ensemble" the full pipeline puting all the components together:

In [10]:
# Build up the pipeline

pipeline = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    clinical_pos,
    clinical_spell
  ])

### Step 4. Generate the model and transform the data
The next step is to generate the model by fiting our content extracted from the PDF into our Spark Dataframe (data):

In [11]:
# We convert the pipeline into a model

model = pipeline.fit(data)

Finally we transform our data using the model:

In [12]:
output = model.transform(data)

### Step 5. Explore our results
We are now ready to explore the output generated by the transformation. For example here is the list of identified sentences and the Part-Of-Speech assigned to each of their tokens:

In [13]:
nrows = output.count()
all_sentences = output.select('sentence.result').take(nrows)
all_tokens = output.select('token.result').take(nrows)
all_pos = output.select('pos.result').take(nrows)

html_output = ""

for row_index in range(nrows):
    html_output += "<div style='font-size: 1.2em; margin: 15px'>"
    
    this_tokens = all_tokens[row_index]['result']
    this_pos = all_pos[row_index]['result']
    
    html_output += "<table>"
    
    html_output += "<tr style='text-align: center'>"
    for token_index in range(len(this_tokens)):
            html_output += "<td style='text-align: center'>" + this_tokens[token_index] + "</td>"
    html_output += "</tr>"
    
    html_output += "<tr>"
    for pos_index in range(len(this_pos)):
            html_output += "<td style='text-align: center'>" + this_pos[pos_index] + "</td>"
    html_output += "</tr>"
    
    html_output += "</table>"
    html_output += "</div>"
   
from IPython.core.display import display, HTML
display(HTML(html_output))

### Step 6. LightPipelines 

LightPipelines allow easier and faster testing for small tests or real-time requests

We will create a LightPipeline using our generated model, and then we will annotate a new text with some typos to see how the SpellChecker deals with them.

In [14]:
light_pipeline = LightPipeline(model)

In [15]:
# Call annotate() in order to test a sentence or a list of sentences
light_data = light_pipeline.annotate("Parcial ressection of the rigth lungg.")

In [16]:
html_output = "<center><table style='font-size: 1.2em'>"
html_output += "<tr><td>Original: </td>"
for this_index in range(len(light_data['token'])):
    html_output += "<td>" + light_data['token'][this_index] + "</td>"
html_output += "</tr>"

html_output += "<tr><td>Fixed: </td>"
for this_index in range(len(light_data['checked'])):
    html_output += "<td>" + light_data['checked'][this_index] + "</td>"
html_output += "</tr>"

html_output += "</table></center>"

from IPython.core.display import display, HTML
display(HTML(html_output))


Original:,Parcial,ressection,of,the,rigth,lungg,.
Fixed:,Partial,resection,of,the,right,lung,.
